In [69]:
import pandas as pd
import numpy as np
from category_encoders import TargetEncoder, CountEncoder

In [158]:
df = pd.read_csv('Datasets/final_dataset.csv')
df = df.drop(['Year_y'],axis=1)
df = df.rename(columns={'Year_x': 'Year'})
df = df[df['Course'].notnull()]
df.Year = df.Year.astype(int)

In [159]:
locations = pd.read_excel('Datasets/School locations.xlsx')

In [160]:
locations = locations[['Customer heading', 'Institution', 'Titularity']]

In [161]:
df = pd.merge(df, locations, on='Customer heading', how='left')

In [162]:
trends = pd.read_csv('Datasets/trends.csv')
trends = trends.rename(columns={'R15_16': 2016, 'R16_17': 2017, 'R17_18': 2018, 'R18_19': 2019})

In [163]:
trends.head()

,Subject,Course,2016,2017,2018,2019
0,spanish,primary 1,102.965458,105.397134,105.537104,109.778895
1,spanish,primary 2,113.567547,113.082270,107.370150,113.145457
2,spanish,primary 3,109.002650,107.737870,104.515799,106.660135
3,spanish,primary 4,110.910233,115.081936,113.583410,106.402572
4,spanish,primary 5,103.490316,106.571920,110.719428,111.567043


In [164]:
def impute_market_index(row):
    
    subject = row.Subject
    course = row.Course
    year = row.Year
    
    if year == 2015:
        market_index = 100
    
    else:
        market_index = trends.loc[(trends.Subject == subject) & (trends.Course == course), year].values[0]
        
    return market_index

In [165]:
df['Market index'] = df.apply(lambda row: impute_market_index(row), axis=1)

In [166]:
df.loc[df.Year.isin([2015,2016,2017]), 'No info on mkt / activities'] = 1
df.loc[df.Year.isin([2018, 2019]), 'No info on mkt / activities'] = 0

In [167]:
df = df.drop('Academic_Year', axis=1)

In [168]:
df = df.rename(columns={'Count_Activities': 'Count activities', 
                   'number_items': 'Number of items', 
                   'returns': 'Returns', 
                   'action_delay_1': 'Action delay 1', 
                   'action_delay_2': 'Action delay 2', 
                   'action_delay_3': 'Action delay 3', 
                   'First Day': 'First day', 
                   'Last Day': 'Last day', 'ADMINISTRATION': 'Administration',
                   'BIOLOGY AND GEOLOGY': 'Biology and Geology', 'ECONOMY': 'Economy', 
                   'ENGLISH (INFANT)': 'English (infant)', 
                   'ENGLISH (PRIMARY)': 'English (primary)', 
                   'ENGLISH (SECONDARY)': 'English (secondary)', 
                   'FRENCH (PRIMARY)': 'French (primary)', 
                   'FRENCH (SECONDARY)': 'French (secondary)', 
                   'FRENCH HIGH SCHOOL': 'French (high school)', 
                   'GEOGRAPHY AND HISTORY': 'Geography and History', 
                   'INFANT': 'Infant', 
                   'INFORMATION TECHNOLOGY': 'Information Technology', 
                   'LATIN & GREEK': 'Latin and Greek', 
                   'LIBRARY': 'Library', 'MANAGEMENT BOARD': 'Management Board', 
                   'MATHEMATICS': 'Mathematics', 'MUSIC (PRIMARY)':'Music (primary)', 
                   'MUSIC (SECONDARY)': 'Music (secondary)', 'ORIENTATION': 'Orientation', 
                   'OTHER':'Other', 'OWNERSHIP TEAM': 'Ownership team', 
                   'PASTORAL TEAM': 'Pastoral team', 'PHILOSOPHY': 'Philosophy', 
                   'PHYSICS AND CHEMISTRY': 'Physics and Chemistry', 'PLASTIC COURSE': 'Plastic course', 
                   'PRIMARY': 'Primary', 'QUALITY': 'Quality', 'REGIONAL LANGUAGE': 'Regional language', 
                   'RELIGION (INFANT)': 'Religion (infant)', 'RELIGION (PRIMARY)': 'Religion (primary)', 
                   'RELIGION (SECONDARY)': 'Religion (secondary)', 'SCIENCE': 'Science', 
                   'SPANISH LANGUAGE': 'Spanish language', 'TECHNOLOGY': 'Technology', 'UNALLOCATED': 'Unallocated'})

In [169]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45020 entries, 0 to 45019
Data columns (total 89 columns):
Year                                      45020 non-null int64
Course                                    45020 non-null object
Subject                                   45020 non-null object
# copies                                  45020 non-null float64
Customer heading                          45020 non-null int64
Count activities                          45020 non-null float64
churn                                     29688 non-null float64
Num customers of seller                   45020 non-null float64
Area covered by seller                    45020 non-null float64
Seller portfolio - Catholic (abs)         45020 non-null float64
Seller portfolio - Private (abs)          45020 non-null float64
Seller portfolio - Catholic (relative)    45020 non-null float64
Seller portfolio - Private (relative)     45020 non-null float64
Institution = Yes % (relative)            45020 non-

In [170]:
test = df[df.Year == 2019].drop('churn', axis=1).fillna(0)

In [171]:
train = df[df.Year != 2019].fillna(0)

In [174]:
te = TargetEncoder(cols=['Institution', 'Titularity'])
te.fit(train.drop('churn', axis=1), train.churn)

TargetEncoder(cols=['Institution', 'Titularity'], drop_invariant=False,
              handle_missing='value', handle_unknown='value',
              min_samples_leaf=1, return_df=True, smoothing=1.0, verbose=0)

In [175]:
train[['Institution', 'Titularity']] = te.transform(train.drop('churn', axis=1))[['Institution', 'Titularity']]

In [176]:
train.to_csv('train_set_cat.csv', index=False)

In [177]:
test[['Institution', 'Titularity']] = te.transform(test)[['Institution', 'Titularity']]

In [178]:
test.to_csv('test_set_cat.csv', index=False)